### Import packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from matplotlib.ticker import AutoLocator, AutoMinorLocator, LogLocator
import glob
from scipy.interpolate import griddata
from pathlib import Path
import h5py
import sys
from pathlib import Path

# Where am I running?
try:
    # Normal script
    here = Path(__file__).resolve().parent
except NameError:
    # Notebook / REPL
    here = Path.cwd()

phys_const_path = (here / '..' / 'phys_const').resolve()
sys.path.append(str(phys_const_path))

nsm_plots_path = (here / '..' / 'nsm_plots').resolve()
sys.path.append(str(nsm_plots_path))

nsm_plots_postproc = (here / '..' / 'nsm_instabilities').resolve()
sys.path.append(str(nsm_plots_postproc))

import phys_const as pc
import plot_functions as pf
import functions_angular_crossings as fac

### File path

In [ ]:
##################################################################################################################
# Simulation paths

direct = '/pscratch/sd/u/uo1999/gw170817_1.00Ye/classical_transport/1506_ppEb'
parfile = '/plt00384_particles'
ncell = (96, 96, 64) # number of cells in each direction
domain = (96e5, 96e5, 64e5) # cm
n_ang_bins = 1506

##################################################################################################################

sim_id = direct.split('/')[-1]  # Get the last part of the directory path

# Calculate the volume of a single cell and the total volume
cellvolume = (domain[0] / ncell[0]) * (domain[1] / ncell[1]) * (domain[2] / ncell[2]) # cm^3
domainvolume = cellvolume * (ncell[0] * ncell[1] * ncell[2]) # cm^3

finaldir = direct.split('/')[-1]
file = '/allData.h5'

print(f'Loading data from {direct}{file}')

### Define cell index to be analyzed

In [ ]:
cell_index_i = 15
cell_index_j = 48
cell_index_k = 15

dx_cell = domain[0] / ncell[0]
dy_cell = domain[1] / ncell[1]
dz_cell = domain[2] / ncell[2]

x_cell_center = (cell_index_i + 0.5) * dx_cell
y_cell_center = (cell_index_j + 0.5) * dy_cell
z_cell_center = (cell_index_k + 0.5) * dz_cell

bh_radius   = 05.43e+05 # cm
bh_center_x = 48.0e+5 # cm
bh_center_y = 48.0e+5 # cm
bh_center_z = 16.0e+5 # cm

# compute rhat_out at the cell center
r_out = np.array([x_cell_center - bh_center_x, y_cell_center - bh_center_y, z_cell_center - bh_center_z])
r_out_mag = np.linalg.norm(r_out)
rhat_out = r_out / r_out_mag

gamma_bh_shadow = np.arcsin(bh_radius / r_out_mag)

r_out_theta = np.arccos(rhat_out[2])
r_out_phi = np.arctan2(rhat_out[1], rhat_out[0])
r_out_phi = np.mod(r_out_phi, 2*np.pi)

print(f'r_out_theta = {r_out_theta * 180.0 / np.pi}')
print(f'r_out_phi = {r_out_phi * 180.0 / np.pi}')

def Rz(phi):
    """Rotation matrix about the z-axis by angle phi (radians)."""
    return np.array([
        [np.cos(phi), -np.sin(phi), 0.0],
        [np.sin(phi),  np.cos(phi), 0.0],
        [0.0,          0.0,         1.0]
    ])

def Ry(phi):
    """Rotation matrix about the y-axis by angle phi (radians)."""
    return np.array([
        [np.cos(phi), 0.0, np.sin(phi)],
        [0.0,         1.0, 0.0],
        [-np.sin(phi),0.0, np.cos(phi)]
    ])

def rotate_theta_phi(theta, phi, rotangle_z, rotangle_y):
    """
    Rotate theta and phi arrays or scalars to new coordinates such that (theta, phi) aligns with north pole.
    Accepts broadcastable arrays theta, phi of any shape.
    
    Returns arrays of theta_rotated, phi_rotated of the same shape.
    """
    # Broadcast theta and phi to common shape
    print(f'function theta.shape = {theta.shape}')
    print(f'function phi.shape = {phi.shape}')

    theta, phi = np.broadcast_arrays(theta, phi)
    orig_shape = theta.shape

    # Flatten for vectorized computation
    theta_flat = theta.ravel()
    phi_flat = phi.ravel()
    
    # Compute rhat vectors, shape (N, 3)
    x = np.sin(theta_flat) * np.cos(phi_flat)
    y = np.sin(theta_flat) * np.sin(phi_flat)
    z = np.cos(theta_flat)
    rhat = np.stack([x, y, z], axis=1)  # (N, 3)

    # Prepare rotated vectors array
    rhat_rot = np.zeros_like(rhat)

    for i in range(rhat.shape[0]):
        # Rotation matrices for current angles (scalar)
        Rz_matrix = Rz(rotangle_z)
        Ry_matrix = Ry(rotangle_y)
        rtmp = Rz_matrix @ rhat[i]
        rrot = Ry_matrix @ rtmp
        rhat_rot[i] = rrot

    # Extract new theta, phi for each rotated vector
    z_rot = rhat_rot[:, 2]
    y_rot = rhat_rot[:, 1]
    x_rot = rhat_rot[:, 0]
    theta_rotated = np.arccos(z_rot)
    phi_rotated = np.arctan2(y_rot, x_rot)
    phi_rotated = np.mod(phi_rotated, 2*np.pi)

    # Reshape back to input
    theta_rotated = theta_rotated.reshape(orig_shape)
    phi_rotated = phi_rotated.reshape(orig_shape)
    return theta_rotated, phi_rotated

# testing
rotate_theta_phi(
    theta = np.array([0.0]),
    phi = np.array([0.0]),
    rotangle_z = -np.pi/2.0,
    rotangle_y = -np.pi/2.0
)
rotate_theta_phi(
    theta = np.array([r_out_theta ]),
    phi = np.array([r_out_phi ]),
    rotangle_z = np.pi - r_out_phi,
    rotangle_y = -np.pi/2.0 + r_out_theta
)

### Cell to be analized

In [ ]:
# Get the cell indices
cell_file_names = glob.glob(direct + '/' + parfile + '/cell_*_*_*')
cell_file_names = [file_name.split('/')[-1] for file_name in cell_file_names]
x_cell_ind = np.array([int(file_name.split('_')[1]) for file_name in cell_file_names])
y_cell_ind = np.array([int(file_name.split('_')[2]) for file_name in cell_file_names])
z_cell_ind = np.array([int((file_name.split('_')[3]).split('.')[0]) for file_name in cell_file_names])
cell_indices = np.array(list(zip(x_cell_ind, y_cell_ind, z_cell_ind)))
print('Number of cells:', len(cell_indices))
print(f'shape of cell_indices: {cell_indices.shape}')

# Get the cell indices for ix fix but iy and iz varying all available cells
x_idx_slice = cell_index_i
mask_yz_slice = cell_indices[:,0] == x_idx_slice # fixing the x index in this value
cell_indices_yz_slice = cell_indices[mask_yz_slice]
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(cell_indices_yz_slice[:,1], cell_indices_yz_slice[:,2], color='b')
ax.scatter(cell_index_j, cell_index_k, color='r')
ax.set_xlabel('$i_y$')
ax.set_ylabel('$i_z$')
ax.set_title(f'Cell indices for fixed $i_x=${x_idx_slice}')
ax.legend()
# ax.set_xlim([-5,100])
# ax.set_ylim([-5,70])
plt.show()
plt.close(fig)

# Get the cell indices for iy fix but ix and iz varying all available cells
y_idx_slice = cell_index_j
mask_xz_slice = cell_indices[:,1] == y_idx_slice # fixing the y index in this value
cell_indices_xz_slice = cell_indices[mask_xz_slice]
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(cell_indices_xz_slice[:,0], cell_indices_xz_slice[:,2], color='b')
ax.scatter(cell_index_i, cell_index_k, color='r')
ax.set_xlabel('$i_x$')
ax.set_ylabel('$i_z$')
ax.set_title(f'Cell indices for fixed $i_y=${y_idx_slice}')
ax.legend()
# ax.set_xlim([-5,100])
# ax.set_ylim([-5,70])
plt.show()
plt.close(fig)

# Get the cell indices for iz fix but ix and iy varying all available cells
z_idx_slice = cell_index_k
mask_xy_slice = cell_indices[:,2] == z_idx_slice # fixing the z index in this value
cell_indices_xy_slice = cell_indices[mask_xy_slice]
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(cell_indices_xy_slice[:,0], cell_indices_xy_slice[:,1], color='b')
ax.scatter(cell_index_i, cell_index_j, color='r')
ax.set_xlabel('$i_x$')
ax.set_ylabel('$i_y$')
ax.set_title(f'Cell indices for fixed $i_z=${z_idx_slice}')
ax.legend()
# ax.set_xlim([-5,100])
# ax.set_ylim([-5,70])
plt.show()
plt.close(fig)
     
# Combine all cell indices from the three slices
# cell_indices_all = np.concatenate((cell_indices_yz_slice, cell_indices_xz_slice, cell_indices_xy_slice), axis=0)
cell_indices_all = np.concatenate((cell_indices_xz_slice, cell_indices_xy_slice), axis=0)
cell_indices_all = np.unique(cell_indices_all, axis=0)
# cell_indices_all = np.array([[0,0,0]])

### Compute ELN number density

In [ ]:
# Dictionary keys
# <KeysViewHDF5 ['N00_Re', 'N00_Rebar', 'N01_Im', 'N01_Imbar', 'N01_Re', 'N01_Rebar', 'N02_Im', 'N02_Imbar', 'N02_Re', 'N02_Rebar', 'N11_Re', 'N11_Rebar', 'N12_Im', 'N12_Imbar', 'N12_Re', 'N12_Rebar', 'N22_Re', 'N22_Rebar', 'TrHN', 'Vphase', 'pos_x', 'pos_y', 'pos_z', 'pupt', 'pupx', 'pupy', 'pupz', 'time', 'x', 'y', 'z']>

def compute_eln_xln(cell_index_i, cell_index_j, cell_index_k):

    particles_dict_this_cell = fac.load_particle_data(cell_index_i, cell_index_j, cell_index_k, direct + '/' + parfile)

    px = particles_dict_this_cell['pupx']/particles_dict_this_cell['pupt']
    py = particles_dict_this_cell['pupy']/particles_dict_this_cell['pupt']
    pz = particles_dict_this_cell['pupz']/particles_dict_this_cell['pupt']
    print(f'pz.shape = {pz.shape}')

    momentum = np.stack((px, py, pz), axis=-1)
    tolerance = 1.0e-4
    unique_momentum = fac.get_unique_momentum(momentum, tolerance)
    print(f'unique_momentum.shape = {unique_momentum.shape}')

    theta = np.arccos(unique_momentum[:,2])
    phi = np.arctan2(unique_momentum[:,1], unique_momentum[:,0])
    phi = np.where(phi < 0, phi + 2 * np.pi, phi) # Adjusting the angle to be between 0 and 2pi.

    nee_all     = particles_dict_this_cell['N00_Re']    / cellvolume
    nuu_all     = particles_dict_this_cell['N11_Re']    / cellvolume
    ntt_all     = particles_dict_this_cell['N22_Re']    / cellvolume
    neebar_all  = particles_dict_this_cell['N00_Rebar'] / cellvolume
    nuubar_all  = particles_dict_this_cell['N11_Rebar'] / cellvolume
    nttbar_all = particles_dict_this_cell['N22_Rebar']  / cellvolume

    fluxee_all    = nee_all   [:, np.newaxis] * momentum
    fluxuu_all    = nuu_all   [:, np.newaxis] * momentum
    fluxtt_all    = ntt_all   [:, np.newaxis] * momentum
    fluxeebar_all = neebar_all[:, np.newaxis] * momentum
    fluxuubar_all = nuubar_all[:, np.newaxis] * momentum
    fluxnttbar_all = nttbar_all[:, np.newaxis] * momentum

    ee_unique_fluxes, ee_unique_fluxes_mag       = fac.compute_unique_fluxes(momentum, fluxee_all, unique_momentum)
    uu_unique_fluxes, uu_unique_fluxes_mag       = fac.compute_unique_fluxes(momentum, fluxuu_all, unique_momentum)
    tt_unique_fluxes, tt_unique_fluxes_mag       = fac.compute_unique_fluxes(momentum, fluxtt_all, unique_momentum)
    eebar_unique_fluxes, eebar_unique_fluxes_mag = fac.compute_unique_fluxes(momentum, fluxeebar_all, unique_momentum)
    uubar_unique_fluxes, uubar_unique_fluxes_mag = fac.compute_unique_fluxes(momentum, fluxuubar_all, unique_momentum)
    ttbar_unique_fluxes, ttbar_unique_fluxes_mag = fac.compute_unique_fluxes(momentum, fluxnttbar_all, unique_momentum)

    eln_xln = (
        (ee_unique_fluxes_mag - eebar_unique_fluxes_mag ) -
        (uu_unique_fluxes_mag - uubar_unique_fluxes_mag ) -  
        (tt_unique_fluxes_mag - ttbar_unique_fluxes_mag ) 
    )    

    return theta, phi, eln_xln

dOmega = 4*np.pi / n_ang_bins  

### Estimate FFI growth rate base on Box3D prediction

In [ ]:
sigma = 0.0
GnPos = 0.0
GnNeg = 0.0

theta, phi, eln_xln = compute_eln_xln(cell_index_i, cell_index_j, cell_index_k)

G_ELN = np.sqrt(2) * pc.PhysConst.GF * ( pc.PhysConst.hbarc**3 / pc.PhysConst.hbar ) * eln_xln

mask_pos_eln = G_ELN > 0
mask_neg_eln = G_ELN < 0

print(f'Number of positive G_ELN: {np.sum(mask_pos_eln)}')
print(f'Number of negative G_ELN: {np.sum(mask_neg_eln)}\n')

sum_pos_eln = np.sum(G_ELN[mask_pos_eln])
sum_neg_eln = -1.0*np.sum(G_ELN[mask_neg_eln])

GnPos = sum_pos_eln
GnNeg = sum_neg_eln

print('Sum of positive G_ELN: {:.3e} 1/s'.format(sum_pos_eln))
print('Sum of negative G_ELN: {:.3e} 1/s\n'.format(sum_neg_eln))

tau = 1.0 / np.sqrt(sum_pos_eln*sum_neg_eln) # seconds
sigma = np.sqrt(sum_pos_eln*sum_neg_eln)

print('sigma: {:.3e} 1/s'.format(sigma))
print('sigma/c: {:.3e} 1/cm\n'.format(sigma / pc.PhysConst.c))

print('tau: {:.3e} s'.format(tau))
print('c * tau: {:.3e} cm'.format(pc.PhysConst.c * tau))

In [ ]:
# Minimums
min_Geln_fi   = -4.05351017e9   # lowest of min_Geln_fi
min_Ge_fp     =  3.52654070e9   # lowest of min_Ge_fp
min_Gebar_fp  =  6.70209233e8   # lowest of min_Gebar_fp
min_Gx_fp     =  1.86383209e6   # lowest of min_Gx_fp

# Maximums
max_Geln_fi   =  4.51833597e9   # highest of max_Geln_fi
max_Ge_fp     =  9.52227218e9   # highest of max_Ge_fp
max_Gebar_fp  =  1.21471491e10  # highest of max_Gebar_fp
max_Gx_fp     =  2.30527141e8   # highest of max_Gx_fp

### Plot ELN-uLN-tLN angular distribution

In [ ]:
# direction going outwards the bh center is +z axis

for cell_index_i, cell_index_j, cell_index_k in [[15,48,15]]:

    theta, phi, eln_xln = compute_eln_xln(cell_index_i, cell_index_j, cell_index_k)

    theta_rotated, phi_rotated = rotate_theta_phi(theta, phi, -r_out_phi, -r_out_theta)
    phi_bh_shadow = np.linspace(0.0, 2.0*np.pi, 100)
    theta_bh_shadow = np.ones_like(phi_bh_shadow) * gamma_bh_shadow

    # direction going outwards the bh center is -x axis
    # theta_rotated, phi_rotated = rotate_theta_phi(theta, phi, np.pi - r_out_phi, -np.pi/2.0 + r_out_theta)
    # theta_bh_shadow, phi_bh_shadow = rotate_theta_phi(theta_bh_shadow, phi_bh_shadow, 0.0, r_out_theta)
    # theta_bh_shadow, phi_bh_shadow = rotate_theta_phi(theta_bh_shadow, phi_bh_shadow, r_out_phi, 0.0)
    # theta_bh_shadow, phi_bh_shadow = rotate_theta_phi(theta_bh_shadow, phi_bh_shadow, np.pi - r_out_phi, -np.pi/2.0 + r_out_theta)

    # compute plot quantities
    phi_fi, mu_fi, eln_xln_fi, phi_for_plot, mu_for_plot, eln_xln_for_plot = fac.do_interpolation(phi_rotated, theta_rotated, eln_xln)

    Geln_fi = np.sqrt(2) * pc.PhysConst.GF * ( pc.PhysConst.hbarc**3 / pc.PhysConst.hbar ) * eln_xln_fi /  dOmega
    Geln_fp = np.sqrt(2) * pc.PhysConst.GF * ( pc.PhysConst.hbarc**3 / pc.PhysConst.hbar ) * eln_xln_for_plot / dOmega

    max_Geln_fi = np.abs(np.nanmax(Geln_fi))
    min_Geln_fi = np.abs(np.nanmin(Geln_fi))
    limits_colorbar = np.min((max_Geln_fi, min_Geln_fi))
    print("max_Geln_fi = {:.8e}".format(np.nanmax(Geln_fi)))
    print("min_Geln_fi = {:.8e}".format(np.nanmin(Geln_fi)))

    pf.plot_pcolormesh_with_contour_and_scatter_one_cbar(
        x1=phi_for_plot,
        y1=mu_for_plot,
        z1=Geln_fp,
        # min_cb1=-limits_colorbar,
        # max_cb1=+limits_colorbar,
        min_cb1=-max_Geln_fi/1e9,
        max_cb1=+max_Geln_fi/1e9,
        # min_cb1=-4.51833597e+09,
        # max_cb1=+4.51833597e+09,
        # min_cb1=None,
        # max_cb1=None,
        # min_cb1=-1e+06,
        # max_cb1=+1e+06,
        cbar_label1=r'$G_{\nu_e}-G_{\bar{\nu}_e}\,\left[\mathrm{s}^{-1}\,\mathrm{Sr}^{-1}\right]$',
        colormap1='bwr',
        x_label=r'$\phi$',
        y_label=r'$\cos\theta$',
        title=f'r=({((cell_index_i+0.5)*dx_cell - bh_center_x) / 1e5}, {((cell_index_j+0.5)*dy_cell - bh_center_y) / 1e5}, {((cell_index_k+0.5)*dz_cell - bh_center_z) / 1e5}) km', 
        filename=f'plots/{sim_id}_eln_angular_dist_cell_{cell_index_i}_{cell_index_j}_{cell_index_k}.png',
        x_scatter1=phi_fi,
        y_scatter1=mu_fi,
        z_scatter1=Geln_fi,
        doshow=True,
        dosave=True
    )

    pf.plot_pcolormesh_with_contour_and_scatter_one_cbar_with_bh_shadow(
        x1=phi_for_plot,
        y1=mu_for_plot,
        z1=np.zeros_like(Geln_fp),
        # z1=Geln_fp,
        # min_cb1=-limits_colorbar,
        # max_cb1=+limits_colorbar,
        min_cb1=-max_Geln_fi/1e9,
        max_cb1=+max_Geln_fi/1e9,
        # min_cb1=-4.51833597e+09,
        # max_cb1=+4.51833597e+09,
        # min_cb1=None,
        # max_cb1=None,
        # min_cb1=-1e+06,
        # max_cb1=+1e+06,
        cbar_label1=r'$G_{\nu_e}-G_{\bar{\nu}_e}\,\left[\mathrm{s}^{-1}\,\mathrm{Sr}^{-1}\right]$',
        colormap1='bwr',
        x_label=r'$\phi$',
        y_label=r'$\cos\theta$',
        title=f'r=({((cell_index_i+0.5)*dx_cell - bh_center_x) / 1e5}, {((cell_index_j+0.5)*dy_cell - bh_center_y) / 1e5}, {((cell_index_k+0.5)*dz_cell - bh_center_z) / 1e5}) km', 
        filename=f'plots/{sim_id}_eln_angular_dist_cell_{cell_index_i}_{cell_index_j}_{cell_index_k}.png',
        x_scatter1=phi_fi,
        y_scatter1=mu_fi,
        z_scatter1=Geln_fi,
        cos_theta_bh_shadow=np.cos(theta_bh_shadow),
        phi_bh_shadow=phi_bh_shadow,
        costheta_lowlimit=np.cos(gamma_bh_shadow)-0.01,
        costheta_highlimit=1.01,
        doshow=True,
        dosave=True
    )


### Plot electron neutrino and antineutrino angular distribution

In [ ]:
phi_fi, mu_fi, ee_fi, phi_for_plot, mu_for_plot, ee_fp = fac.do_interpolation(phi, theta, ee_unique_fluxes_mag)
Ge_fi = np.sqrt(2) * pc.PhysConst.GF * ( pc.PhysConst.hbarc**3 / pc.PhysConst.hbar ) * ee_fi /  dOmega
Ge_fp = np.sqrt(2) * pc.PhysConst.GF * ( pc.PhysConst.hbarc**3 / pc.PhysConst.hbar ) * ee_fp / dOmega

phi_fi, mu_fi, eebar_fi, phi_for_plot, mu_for_plot, eebar_for_plot = fac.do_interpolation(phi, theta, eebar_unique_fluxes_mag)
Gebar_fi = np.sqrt(2) * pc.PhysConst.GF * ( pc.PhysConst.hbarc**3 / pc.PhysConst.hbar ) * eebar_fi /  dOmega
Gebar_fp = np.sqrt(2) * pc.PhysConst.GF * ( pc.PhysConst.hbarc**3 / pc.PhysConst.hbar ) * eebar_for_plot / dOmega

Combined_min = np.nanmin(np.stack((Ge_fp, Gebar_fp)))
Combined_max = np.nanmax(np.stack((Ge_fp, Gebar_fp)))

print(f"Combined_min = {Combined_min:.8e}")
print(f"Combined_max = {Combined_max:.8e}")

pf.plot_pcolormesh_with_contour_and_scatter_one_cbar(
    x1=phi_for_plot,
    y1=mu_for_plot,
    z1=Ge_fp,
    # min_cb1=min_Ge_fp,
    # max_cb1=max_Ge_fp,
    # min_cb1=None,
    # max_cb1=None,
    min_cb1=Combined_min,
    max_cb1=Combined_max,
    cbar_label1=r'$G_{\nu_e}\,\left[\mathrm{s}^{-1}\,\mathrm{Sr}^{-1}\right]$',
    colormap1='viridis',
    x_label=r'$\phi$',
    y_label=r'$\cos\theta$',
    title=f'', 
    filename=f'plots/{sim_id}_e_angular_dist_cell_{cell_index_i}_{cell_index_j}_{cell_index_k}.png',
    x_scatter1=phi_fi,
    y_scatter1=mu_fi,
    z_scatter1=Ge_fi
)


pf.plot_pcolormesh_with_contour_and_scatter_one_cbar(
    x1=phi_for_plot,
    y1=mu_for_plot,
    z1=Gebar_fp,
    # min_cb1=min_Gebar_fp,
    # max_cb1=max_Gebar_fp,    
    # min_cb1=None,
    # max_cb1=None,
    min_cb1=Combined_min,
    max_cb1=Combined_max,
    cbar_label1=r'$G_{\bar{\nu}_e}\,\left[\mathrm{s}^{-1}\,\mathrm{Sr}^{-1}\right]$',
    colormap1='viridis',
    x_label=r'$\phi$',
    y_label=r'$\cos\theta$',
    title=f'', 
    filename=f'plots/{sim_id}_ebar_angular_dist_cell_{cell_index_i}_{cell_index_j}_{cell_index_k}.png',
    x_scatter1=phi_fi,
    y_scatter1=mu_fi,
    z_scatter1=Gebar_fi
)

phi_fi, mu_fi, xx_fi, phi_for_plot, mu_for_plot, xx_for_plot = fac.do_interpolation(phi, theta, uu_unique_fluxes_mag)
Gx_fi = np.sqrt(2) * pc.PhysConst.GF * ( pc.PhysConst.hbarc**3 / pc.PhysConst.hbar ) * xx_fi /  dOmega
Gx_fp = np.sqrt(2) * pc.PhysConst.GF * ( pc.PhysConst.hbarc**3 / pc.PhysConst.hbar ) * xx_for_plot / dOmega

pf.plot_pcolormesh_with_contour_and_scatter_one_cbar(
    x1=phi_for_plot,
    y1=mu_for_plot,
    z1=Gx_fp,
    # min_cb1=min_Gx_fp,
    # max_cb1=max_Gx_fp,
    min_cb1=None,
    max_cb1=None,
    cbar_label1=r'$G_{\nu_e}\,\left[\mathrm{s}^{-1}\,\mathrm{Sr}^{-1}\right]$',
    colormap1='viridis',
    x_label=r'$\phi$',
    y_label=r'$\cos\theta$',
    title=f'', 
    filename=f'plots/{sim_id}_x_angular_dist_cell_{cell_index_i}_{cell_index_j}_{cell_index_k}.png',
    x_scatter1=phi_fi,
    y_scatter1=mu_fi,
    z_scatter1=Gx_fi
)

print("max_Ge_fp = {:.8e}".format(np.nanmax(Ge_fp)))
print("min_Ge_fp = {:.8e}".format(np.nanmin(Ge_fp)))

print("max_Gebar_fp = {:.8e}".format(np.nanmax(Gebar_fp)))
print("min_Gebar_fp = {:.8e}".format(np.nanmin(Gebar_fp)))

print("max_Gx_fp = {:.8e}".format(np.nanmax(Gx_fp)))
print("min_Gx_fp = {:.8e}".format(np.nanmin(Gx_fp)))